In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

In [ ]:
# Extract from file
file_name = 'BTC-USD.csv'
file = 'data/'+file_name
dataset = pd.read_csv(file)

max = np.amax(dataset['Close'])
min = np.amin(dataset['Close'])

dataset = dataset.apply(lambda x: (x-min)/(max-min) if x.name == 'Close' else x, axis = 0)

In [ ]:
def get_number_for_date(dataset, date):
  date_num = dataset.loc[dataset.isin([date]).any(axis=1)].index.values
  if date_num:
    date_num = date_num.item()
  else:
    date_obj = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    first_date = dataset['Date'][0]
    first_date_obj = datetime.strptime(first_date, '%Y-%m-%d %H:%M:%S')
    first_diff = (date_obj - first_date_obj).days
    if first_diff < 0:
      date_num = first_diff
    else:
      last_date = dataset['Date'][len(dataset) - 1]
      last_date_obj = datetime.strptime(last_date, '%Y-%m-%d %H:%M:%S')
      last_diff = (date_obj - last_date_obj).days
      date_num = len(dataset) + last_diff
  return date_num

In [ ]:
def get_window(dataset, window_length, date, flag = False):
  row = get_number_for_date(dataset, date)
  if flag:
    end = row
  else:
    end = row - 1
  start = end - window_length
  window = []
  for i in range(start, end):
    value = 0
    try:
      value = dataset.loc[i]['Close']
    except KeyError as e:
      value = np.average(dataset['Close'])
    window.append(value)
  window = np.array(window)
  window = np.reshape(window, (window_length, -1))
  return window

In [ ]:
def create_model(window_length):
  model = Sequential()
  model.add(Dense(256, activation = 'relu', input_shape = (window_length, 1)))
  model.add(LSTM(256, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = False))
  model.add(Dense(256, activation = 'relu'))
  model.add(Dense(1))
  model.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [ ]:
window_length = 7
num_epochs = 20
dates = []
closes = []
num_samples = len(dataset)
divider = int(num_samples*0.9)
date_tests = []
close_tests = []
test_dates = []
for index, row in dataset.iterrows():
  window = get_window(dataset, window_length, row['Date'])
  close_price = row['Close']
  if index < divider:
    dates.append(window)
    closes.append(close_price)
  else:
    date_tests.append(window)
    close_tests.append(close_price)
    test_dates.append(row['Date'])

X_train = np.array(dates)
Y_train = np.array(closes)

X_test = np.array(date_tests)
Y_test = np.array(close_tests)

model = create_model(window_length)
model.fit(X_train, Y_train, epochs = num_epochs)
_, acc = model.evaluate(X_test, Y_test)
print('Accuracy na test skupu: ', acc)

# Iscrtavanje
fig = plt.figure(figsize=(15, 5))
graph1 = fig.add_subplot(1, 2, 1)

Y_predict = model.predict(X_test)
X_test = np.reshape(X_test,(1,-1))[0]
Y_predict = np.reshape(Y_predict,(1,-1))[0]

Y_predict = (Y_predict*(max-min)+min)
Y_test = (Y_test*(max-min)+min)

graph1.set_xlabel('datum')
graph1.set_ylabel('vrednost')
graph1.plot(test_dates, Y_predict, color='r')
graph1.plot(test_dates, Y_test, color='g')